Welcome to the School Achievement Dashboard! Here, we'll look at various factors to educate decision-makers on how best to invest to improve student learning outcomes.

*Produced by Colin Howard and Luke Moffit for CSCI 77800 -- EthiCS, Fall 2024*

# Part 1. Poverty and Achievement

Start by running all of the cells in this notebook. Then use the dropdown below to choose whether to look at city-, state-, or national-level data.

In [ ]:
#region
# @title Level selector

from ipywidgets import *
import pandas as pd
import urllib.request as rq
import plotly.express as px
import plotly.graph_objects as go
import IPython.display as display
import json

levels = {
    "New York City": "XN",
    "New York State": "NY",
    "United States": "NT"
}

level = "NT"

def set_level(choice):
    level = choice

level_selector = Dropdown(options=levels,description="Level:")
interact(set_level,choice=level_selector);

#endregion

In [ ]:
subjects = ["Mathematics","Reading","Science"]

subject = "Mathematics"

subscales = {
    "Mathematics" : "MRPCM",
    "Reading": "RRPCM",
    "Science": "SRPUV"
}

subscale = "MRPCM"

years = {
    "Mathematics": [1990, 1992, 1996, 1996, 2000, 2000, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Reading": [1992, 1994, 1998, 1998, 2002, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019, 2022][::-1],
    "Science": [2009, 2015, 2019][::-1]
}

year = 2019

grades = {
    "Mathematics": [4,8],
    "Reading": [4,8,12],
    "Science": [4,8,12]
}

grade = 8

variable = "SLUNCH3"

stattype = "MN:MN,PC:P1,PC:P2,PC:P5,PC:P7,PC:P9"

stattypes = {
    "MN:MN": "Mean",
    "PC:P1": "10th Percentile",
    "PC:P2": "25th Percentile",
    "PC:P5": "50th Percentile",
    "PC:P7": "75th Percentile",
    "PC:P9": "90th Percentile"
}

marker_colors = {
    "MN:MN": px.colors.qualitative.Prism[10],
    "PC:P1": px.colors.qualitative.Prism[7],
    "PC:P2": px.colors.qualitative.Prism[5],
    "PC:P5": px.colors.qualitative.Prism[4],
    "PC:P7": px.colors.qualitative.Prism[1],
    "PC:P9": px.colors.qualitative.Prism[0]
}

def build_frame(subject="Mathematics",year=2019, level=level):
    level = level
    year = year
    subject = subject
    subscale = subscales[subject]
    response = rq.urlopen(f"https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data"
                        f"&subject={subject.lower()}"
                        f"&grade={grade}"
                        f"&subscale={subscale}"
                        f"&variable={variable}"
                        f"&jurisdiction={level}"
                        f"&stattype={stattype}"
                        f"&year={year}")
    response_string = response.read().decode('utf-8')
    try:
        df = pd.DataFrame(json.loads(response_string)['result'])
        return df
    except:
        return response_string

def build_figure(df):
    traces = []
    for stat in stattypes.keys():
        trace = go.Bar(
            x=df.loc[df['stattype']==stat]["varValueLabel"], 
            y=df.loc[df['stattype']==stat]["value"], 
            name=stattypes[stat], 
            marker={"color": marker_colors[stat]}
        )
        traces.append(trace)
    return traces

df = build_frame(subject,year)
fw = go.FigureWidget(data=build_figure(df),layout=go.Layout(barmode="group",title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}"))

def update_figure(change):
    subject = subject_widget.value
    year = year_widget.value
    level = level_selector.value
    #what happens when the API complains?
    df = build_frame(subject,year, level)
    with fw.batch_update():
        for i, stat in enumerate(stattypes):
            fw.data[i].x=df.loc[df['stattype']==stat]['varValueLabel']
            fw.data[i].y=df.loc[df['stattype']==stat]['value']
        fw.update_layout(title=f"{grade}th Grade {subject} NAEP Scores and Federal Lunch Program Eligibility, {year}, {dict(zip(levels.values(), levels.keys()))[level]}")
    
subject_widget = Dropdown(options=subjects,description="Subject:")
year_widget = Dropdown(options=years['Mathematics'],description="Year:")

def update_years(*args):
    year_widget.options = years[subject_widget.value]
level_selector.observe(update_figure, 'value')
subject_widget.observe(update_years, 'value')

subject_widget.observe(update_figure, 'value')
year_widget.observe(update_figure, 'value')

container = VBox([HBox([level_selector, subject_widget,year_widget]),fw])
container

# interact(update_figure,subject=subject_widget,year=year_widget);

# Part 2. Poverty and Funding

In [ ]:
import numpy
df = pd.read_csv("FundingData.csv")
#df
#different traces for each state
f = go.FigureWidget()
f.layout.title = "Per Pupil Funding by State over Time"
f.layout.xaxis.title = "Year"
f.layout.yaxis.title = "Funding"
#x = range(2021, 1986, -1)
x = numpy.array(range(2021, 1986, -1))
for state in df["State Name"].unique():
    #x is the year, y is the funding. individual bars are the state
    #y shoulud access the single row of funding for the state
    f.add_scatter(x=x,y=df.loc[df["State Name"] == state].to_numpy()[0][1:],name=state)
    #print(df.loc[df["State Name"] == state].to_numpy())
f

#line graph over time, all 50 states

In [ ]:
#scatterplot that shows school poverty vs funding
#we already know that poverty impacts achievement, from previous graph
#now we want to show the correlation between poverty and funding 
#we are going to do this using data from NCES (?)
from numpy import int64
from pandas import Int64Dtype
import numpy as np

levels2 = {
    "New York State": "NY",
    "United States": "NT"
}

level2 = "NT"

def set_level2(choice):
    level2 = choice

level_selector2 = Dropdown(options=levels2,description="Level:")
#interact(set_level,choice=level_selector2);

#Source: https://nces.ed.gov/ccd/elsi/tableGenerator.aspx
povertyDF = pd.read_csv("DistrictFundingData.csv")
#Source: https://www.census.gov/data/datasets/2022/demo/saipe/2022-school-districts.html
censusDF = pd.read_csv("ussd22.csv")
censusDF["Agency ID - NCES Assigned [District] Latest available year"] = (censusDF["State FIPS Code"].astype(str).str.zfill(2) + censusDF["District ID"].astype(str).str.zfill(5)).astype(int64)
#combine both FIPS ID with the distrcit ID from census data
df = pd.merge(censusDF,povertyDF)
#divide children in poverty by the total population
df["Percentage in poverty"] = df["Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder"].str.replace(",","").astype(int)/df["Estimated Population 5-17"].str.replace(",","").astype(int)*100
#convert 
df = df[df["Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2021-22"].str.isnumeric()]
df["Expenditures per student"] = df["Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2021-22"].astype(int)
f = go.FigureWidget()
f.layout.title = "Per Pupil Funding by Districts vs. Childhood Poverty"
f.layout.xaxis.title = "Percentage of Childhood Poverty"
f.layout.yaxis.title = "Funding"
scatter = go.Scatter(x=df["Percentage in poverty"],y=df["Expenditures per student"],mode="markers",text=df["Agency Name"])
f.add_trace(scatter)



def update_poverty(change):
    level2 = level_selector2.value
    #need to restrict data based on the area
    
    if level2=="NT":    
        tempdf = df
    elif level2=="NY":
        tempdf = df[df["State Abbr [District] Latest available year"].str.contains("NY")]
        print(tempdf)
    elif level2=="XN":
        #NEW YORK CITY per pupil data not available
        tempdf = df[df["Agency Name [District] 2021-22"].str.contains("NEW YORK CITY")]  
        print(tempdf)
    with f.batch_update():
        f.data[0].x=tempdf["Percentage in poverty"]
        f.data[0].y=tempdf["Expenditures per student"]
        f.data[0].text=tempdf["Agency Name"]
        


level_selector2.observe(update_poverty, 'value')

container = VBox([HBox([level_selector2]),f])
container

# Part 3. Factors Affecting Achievement